# TransformerBlock: Assembler Tous les Composants

## Introduction

Bienvenue dans ce notebook sur le **TransformerBlock**, l'unité de base des architectures transformer!

### Objectifs pédagogiques

Dans ce notebook, vous allez:
1. Comprendre l'architecture complète d'un TransformerBlock
2. Assembler Multi-Head Attention, LayerNorm, et Feed-Forward
3. Implémenter les connexions résiduelles (residual connections)
4. Comprendre le rôle de LayerNorm et Dropout
5. Valider la préservation des dimensions
6. Comparer BERT (bidirectionnel) vs GPT (causal)
7. Empiler plusieurs blocs pour créer un transformer profond

### Qu'est-ce qu'un TransformerBlock?

Un TransformerBlock est l'unité de base qui est répétée N fois dans un transformer.

**Composants:**
- **Multi-Head Attention:** Permet aux tokens de s'attendre mutuellement
- **Layer Normalization:** Stabilise l'entraînement
- **Feed-Forward Network:** Transformations non-linéaires
- **Residual Connections:** Permettent au gradient de circuler
- **Dropout:** Régularisation

**Architecture:**
```
Input
  ↓
Multi-Head Attention
  ↓
Add & Norm (residual + LayerNorm)
  ↓
Feed-Forward Network
  ↓
Add & Norm (residual + LayerNorm)
  ↓
Output
```

## 1. Formules Mathématiques du TransformerBlock

### Architecture Complète

Un TransformerBlock applique deux sous-couches avec connexions résiduelles:

#### Sous-Couche 1: Multi-Head Attention

$x' = \text{LayerNorm}(x + \text{Dropout}(\text{MultiHeadAttention}(x)))$

#### Sous-Couche 2: Feed-Forward Network

$\text{output} = \text{LayerNorm}(x' + \text{Dropout}(\text{FFN}(x')))$

### Connexions Résiduelles (Residual Connections)

Les connexions résiduelles permettent au gradient de circuler directement:

$\text{output} = x + F(x)$

Où $F(x)$ est une transformation (attention ou FFN).

**Pourquoi les residual connections?**
- Permettent d'entraîner des réseaux très profonds (>100 couches)
- Le gradient peut circuler directement: $\frac{\partial L}{\partial x} = \frac{\partial L}{\partial \text{output}} + \frac{\partial L}{\partial F(x)}$
- Si $F(x)$ n'apporte rien, le réseau peut l'ignorer (poids → 0)
- Introduites dans ResNet, adoptées par tous les transformers

### Layer Normalization

$\text{LayerNorm}(x) = \gamma \odot \frac{x - \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta$

Où:
- $\mu = \frac{1}{d} \sum_{i=1}^{d} x_i$ : Moyenne sur la dimension des features
- $\sigma^2 = \frac{1}{d} \sum_{i=1}^{d} (x_i - \mu)^2$ : Variance sur la dimension des features
- $\gamma, \beta$ : Paramètres apprenables (scale et shift)
- $\epsilon$ : Petit nombre pour éviter division par zéro (1e-5)

**Différence avec BatchNorm:**
- **BatchNorm:** Normalise sur le batch (dimension 0)
- **LayerNorm:** Normalise sur les features (dimension -1)
- **LayerNorm** fonctionne mieux pour les séquences et petits batches

**Pourquoi LayerNorm?**
- Stabilise l'entraînement en réduisant le "covariate shift"
- Permet d'utiliser des learning rates plus élevés
- Réduit la sensibilité à l'initialisation
- Accélère la convergence

### Dropout

Pendant l'entraînement, chaque neurone a une probabilité $p$ d'être désactivé:

$\text{Dropout}(x)_i = \begin{cases} 
0 & \text{avec probabilité } p \\
\frac{x_i}{1-p} & \text{avec probabilité } 1-p
\end{cases}$

Pendant l'évaluation: $\text{Dropout}(x) = x$ (pas de dropout)

**Pourquoi le Dropout?**
- Régularisation: Prévient le surapprentissage
- Force la redondance: Le réseau ne peut pas dépendre d'un seul neurone
- Effet d'ensemble: Comme entraîner plusieurs réseaux en parallèle

### Dimensions

**Propriété importante:** Le TransformerBlock préserve les dimensions!

- **Input:** $(\text{batch}, \text{seq\_len}, d_{\text{model}})$
- **Output:** $(\text{batch}, \text{seq\_len}, d_{\text{model}})$

Cette propriété permet d'empiler plusieurs blocs séquentiellement.

In [ ]:
# Imports
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from typing import Tuple, Optional

# Ajouter le chemin vers src/
sys.path.append('../..')

# Importer nos modules
from src.architecture.transformer_block import (
    TransformerBlock,
    validate_transformer_block,
    example_transformer_block,
    example_stacked_transformer_blocks
)
from src.architecture.feed_forward import FeedForward
from src.attention.multi_head import MultiHeadAttention
from src.attention.masking import create_causal_mask

# Configuration pour les visualisations
plt.style.use('default')
sns.set_palette("husl")

# Seed pour la reproductibilité
np.random.seed(42)
torch.manual_seed(42)

print("✓ Imports réussis!")
print(f"✓ PyTorch version: {torch.__version__}")
print(f"✓ Device disponible: {'GPU' if torch.cuda.is_available() else 'CPU'}")

## 2. Diagramme d'Architecture

### Visualisation du TransformerBlock

Voici un diagramme détaillé montrant le flux de données à travers un TransformerBlock:

```
┌─────────────────────────────────────────────────────────────┐
│                        Input (x)                             │
│                  (batch, seq_len, d_model)                   │
└──────────────────────────┬──────────────────────────────────┘
                           │
                           ├──────────────────┐
                           │                  │
                           ▼                  │ (residual)
                  ┌────────────────────┐      │
                  │ Multi-Head         │      │
                  │ Attention          │      │
                  └────────┬───────────┘      │
                           │                  │
                           ▼                  │
                  ┌────────────────────┐      │
                  │ Dropout            │      │
                  └────────┬───────────┘      │
                           │                  │
                           ▼                  │
                  ┌────────────────────┐      │
                  │ Add (x + attn)     │◄─────┘
                  └────────┬───────────┘
                           │
                           ▼
                  ┌────────────────────┐
                  │ LayerNorm          │
                  └────────┬───────────┘
                           │ (x')
                           ├──────────────────┐
                           │                  │
                           ▼                  │ (residual)
                  ┌────────────────────┐      │
                  │ Feed-Forward       │      │
                  │ Network            │      │
                  └────────┬───────────┘      │
                           │                  │
                           ▼                  │
                  ┌────────────────────┐      │
                  │ Dropout            │      │
                  └────────┬───────────┘      │
                           │                  │
                           ▼                  │
                  ┌────────────────────┐      │
                  │ Add (x' + ffn)     │◄─────┘
                  └────────┬───────────┘
                           │
                           ▼
                  ┌────────────────────┐
                  │ LayerNorm          │
                  └────────┬───────────┘
                           │
                           ▼
┌─────────────────────────────────────────────────────────────┐
│                       Output                                 │
│                  (batch, seq_len, d_model)                   │
└─────────────────────────────────────────────────────────────┘
```

**Points clés:**
1. **Deux sous-couches:** Attention + FFN
2. **Deux residual connections:** Une pour chaque sous-couche
3. **Deux LayerNorm:** Une après chaque residual connection
4. **Dropout:** Appliqué après attention et FFN
5. **Dimension préservée:** Input et output ont la même shape

## 3. Création d'un TransformerBlock

### 3.1 Instanciation du Module

In [ ]:
# Paramètres
d_model = 256
num_heads = 8
d_ff = 1024  # 4 × d_model
dropout = 0.1

# Créer le TransformerBlock
block = TransformerBlock(
    d_model=d_model,
    num_heads=num_heads,
    d_ff=d_ff,
    dropout=dropout
)

print("=" * 70)
print("TRANSFORMERBLOCK CRÉÉ")
print("=" * 70)
print(f"\nConfiguration:")
print(f"  - d_model: {d_model}")
print(f"  - num_heads: {num_heads}")
print(f"  - d_ff: {d_ff}")
print(f"  - dropout: {dropout}")

print(f"\nArchitecture:")
print(block)

# Compter les paramètres
total_params = sum(p.numel() for p in block.parameters())
print(f"\nNombre total de paramètres: {total_params:,}")

# Détail par composant
print(f"\nDétail des paramètres:")
for name, param in block.named_parameters():
    print(f"  {name}: {param.shape} ({param.numel():,} params)")

### 3.2 Forward Pass Sans Masque (BERT-style)

Testons le bloc avec attention bidirectionnelle (pas de masque).

In [ ]:
# Créer un input de test
batch_size = 2
seq_len = 10
x = torch.randn(batch_size, seq_len, d_model)

print("=" * 70)
print("FORWARD PASS SANS MASQUE (BERT-style)")
print("=" * 70)
print(f"\nInput shape: {x.shape}")
print(f"  (batch_size={batch_size}, seq_len={seq_len}, d_model={d_model})")

# Mode évaluation (désactive dropout)
block.eval()

# Forward pass
with torch.no_grad():
    output = block(x, mask=None)

print(f"\nOutput shape: {output.shape}")
print(f"  (batch_size={batch_size}, seq_len={seq_len}, d_model={d_model})")

print(f"\n✓ Dimensions préservées: {x.shape == output.shape}")

# Statistiques
print(f"\nStatistiques:")
print(f"  Input  - Mean: {x.mean().item():.6f}, Std: {x.std().item():.6f}")
print(f"  Output - Mean: {output.mean().item():.6f}, Std: {output.std().item():.6f}")

### 3.3 Forward Pass Avec Masque Causal (GPT-style)

Testons maintenant avec un masque causal pour l'attention autoregressive.

In [ ]:
print("=" * 70)
print("FORWARD PASS AVEC MASQUE CAUSAL (GPT-style)")
print("=" * 70)

# Créer un masque causal
causal_mask = create_causal_mask(seq_len, x.device)

print(f"\nMasque causal ({seq_len}×{seq_len}):")
print(causal_mask)
print(f"\nInterprétation:")
print(f"  - 1: Attention autorisée")
print(f"  - 0: Attention bloquée (positions futures)")

# Forward pass avec masque
with torch.no_grad():
    output_masked = block(x, mask=causal_mask)

print(f"\nOutput shape: {output_masked.shape}")
print(f"✓ Dimensions préservées: {x.shape == output_masked.shape}")

# Comparer avec et sans masque
diff = torch.abs(output - output_masked).mean().item()
print(f"\nDifférence moyenne entre BERT et GPT: {diff:.6f}")
print(f"  (Les sorties sont différentes car le masque change l'attention)")

## 4. Validation des Dimensions

### 4.1 Shape Checks Détaillés

Vérifions que chaque étape du forward pass préserve les dimensions correctement.

In [ ]:
print("=" * 70)
print("SHAPE CHECKS DÉTAILLÉS")
print("=" * 70)

# Créer un input
x_test = torch.randn(2, 5, d_model)
print(f"\nInput shape: {x_test.shape}")

# Accéder aux composants internes
block.eval()

# Étape 1: Multi-Head Attention
with torch.no_grad():
    attn_output = block.attention(x_test, mask=None)
print(f"\nAprès Multi-Head Attention: {attn_output.shape}")
print(f"  ✓ Shape préservée")

# Étape 2: Dropout
with torch.no_grad():
    attn_dropped = block.dropout(attn_output)
print(f"Après Dropout: {attn_dropped.shape}")
print(f"  ✓ Shape préservée")

# Étape 3: Residual + LayerNorm
with torch.no_grad():
    x_norm1 = block.norm1(x_test + attn_dropped)
print(f"Après Add & Norm 1: {x_norm1.shape}")
print(f"  ✓ Shape préservée")

# Étape 4: Feed-Forward
with torch.no_grad():
    ff_output = block.feed_forward(x_norm1)
print(f"\nAprès Feed-Forward: {ff_output.shape}")
print(f"  ✓ Shape préservée")

# Étape 5: Dropout
with torch.no_grad():
    ff_dropped = block.dropout(ff_output)
print(f"Après Dropout: {ff_dropped.shape}")
print(f"  ✓ Shape préservée")

# Étape 6: Residual + LayerNorm
with torch.no_grad():
    final_output = block.norm2(x_norm1 + ff_dropped)
print(f"Après Add & Norm 2: {final_output.shape}")
print(f"  ✓ Shape préservée")

print(f"\n{'='*70}")
print("✓ Toutes les étapes préservent les dimensions!")
print(f"{'='*70}")

### 4.2 Validation Complète

Utilisons la fonction de validation pour tester différentes configurations.

In [ ]:
# Valider avec différentes configurations
validate_transformer_block(
    d_model=256,
    num_heads=8,
    d_ff=1024,
    batch_size=4,
    seq_len=16,
    dropout=0.1
)

## 5. Comparaison BERT vs GPT

### 5.1 Différence Fondamentale: Le Masque

La principale différence entre BERT et GPT est l'utilisation du masque causal:

**BERT (Bidirectionnel):**
- Pas de masque
- Chaque token peut voir tous les autres tokens
- Utilisé pour des tâches de compréhension (classification, NER, QA)

**GPT (Autorégressif):**
- Masque causal
- Chaque token ne peut voir que les tokens précédents
- Utilisé pour la génération de texte

### 5.2 Démonstration Visuelle

In [ ]:
# Créer un exemple simple
seq_len_demo = 5
x_demo = torch.randn(1, seq_len_demo, 128)

# Créer deux blocs identiques
block_bert = TransformerBlock(d_model=128, num_heads=4, d_ff=512, dropout=0.0)
block_gpt = TransformerBlock(d_model=128, num_heads=4, d_ff=512, dropout=0.0)

# Copier les poids pour une comparaison équitable
block_gpt.load_state_dict(block_bert.state_dict())

block_bert.eval()
block_gpt.eval()

print("=" * 70)
print("BERT vs GPT: Comparaison")
print("=" * 70)

# BERT: Sans masque
with torch.no_grad():
    output_bert = block_bert(x_demo, mask=None)

print(f"\nBERT (bidirectionnel):")
print(f"  - Masque: Aucun")
print(f"  - Chaque token voit: Tous les tokens (0-{seq_len_demo-1})")
print(f"  - Output shape: {output_bert.shape}")

# GPT: Avec masque causal
causal_mask_demo = create_causal_mask(seq_len_demo, x_demo.device)
with torch.no_grad():
    output_gpt = block_gpt(x_demo, mask=causal_mask_demo)

print(f"\nGPT (autorégressif):")
print(f"  - Masque: Causal (triangulaire)")
print(f"  - Token i voit: Tokens 0 à i seulement")
print(f"  - Output shape: {output_gpt.shape}")

# Analyser les différences
print(f"\nAnalyse des différences:")
for pos in range(seq_len_demo):
    diff_pos = torch.abs(output_bert[0, pos] - output_gpt[0, pos]).mean().item()
    print(f"  Position {pos}: différence moyenne = {diff_pos:.6f}")

print(f"\nObservation:")
print(f"  - Les premières positions sont similaires")
print(f"  - Les différences augmentent vers la fin")
print(f"  - Le masque causal change progressivement le comportement")

## 6. Empiler Plusieurs TransformerBlocks

### 6.1 Créer un Stack de Blocs

Un transformer complet est composé de plusieurs TransformerBlocks empilés.

In [ ]:
# Configuration
d_model = 128
num_heads = 4
d_ff = 512
num_layers = 3  # 3 blocs empilés

print("=" * 70)
print(f"STACK DE {num_layers} TRANSFORMERBLOCKS")
print("=" * 70)

# Créer un stack avec nn.ModuleList
blocks = nn.ModuleList([
    TransformerBlock(d_model, num_heads, d_ff, dropout=0.1)
    for _ in range(num_layers)
])

# Compter les paramètres
total_params = sum(p.numel() for p in blocks.parameters())
params_per_block = total_params // num_layers

print(f"\nConfiguration:")
print(f"  - Nombre de blocs: {num_layers}")
print(f"  - d_model: {d_model}")
print(f"  - num_heads: {num_heads}")
print(f"  - d_ff: {d_ff}")

print(f"\nParamètres:")
print(f"  - Total: {total_params:,}")
print(f"  - Par bloc: {params_per_block:,}")

# Créer un input
x = torch.randn(2, 8, d_model)
causal_mask = create_causal_mask(8, x.device)

print(f"\nForward pass à travers le stack:")
print(f"  Input shape: {x.shape}")

# Passer à travers tous les blocs
for i, block in enumerate(blocks):
    block.eval()
    with torch.no_grad():
        x = block(x, mask=causal_mask)
    print(f"  Après bloc {i+1}: {x.shape}")

print(f"\n  Output final: {x.shape}")
print(f"  ✓ Dimension préservée à travers tous les blocs!")

### 6.2 Exemple Complet avec Fonction Utilitaire

In [ ]:
# Utiliser la fonction d'exemple
example_stacked_transformer_blocks()

## 7. Exercices Pratiques

### Exercice 1: Expérimenter avec le Nombre de Têtes

Testez différents nombres de têtes et observez l'impact sur les paramètres.

In [ ]:
# TODO: Complétez ce code
d_model = 256
d_ff = 1024

# Testez différents nombres de têtes
num_heads_list = [1, 2, 4, 8, 16]

print("=" * 70)
print("EXERCICE 1: Impact du nombre de têtes")
print("=" * 70)
print(f"\n{'Têtes':>8} | {'Paramètres':>15} | {'Ratio':>10}")
print("=" * 70)

base_params = None
for num_heads in num_heads_list:
    # TODO: Créez un TransformerBlock et comptez les paramètres
    block = TransformerBlock(d_model, num_heads, d_ff, dropout=0.0)
    total_params = sum(p.numel() for p in block.parameters())
    
    if base_params is None:
        base_params = total_params
    
    ratio = total_params / base_params
    print(f"{num_heads:>8} | {total_params:>15,} | {ratio:>10.2f}x")

print("=" * 70)
print("\nObservation: Le nombre de paramètres ne change pas beaucoup!")
print("Pourquoi? Les têtes partagent les mêmes dimensions totales.")

### Exercice 2: Mesurer l'Impact du Dropout

In [ ]:
# TODO: Mesurez la variance des sorties avec différents taux de dropout
d_model = 128
d_ff = 512
dropout_rates = [0.0, 0.1, 0.3, 0.5]

x = torch.randn(1, 10, d_model)

print("=" * 70)
print("EXERCICE 2: Impact du dropout")
print("=" * 70)

for dropout in dropout_rates:
    block = TransformerBlock(d_model, num_heads=4, d_ff=d_ff, dropout=dropout)
    block.train()  # Mode entraînement pour activer le dropout
    
    # Faire plusieurs forward passes
    outputs = []
    for _ in range(20):
        with torch.no_grad():
            out = block(x, mask=None)
            outputs.append(out)
    
    # Calculer la variance
    outputs_stacked = torch.stack(outputs)
    variance = outputs_stacked.var(dim=0).mean().item()
    
    print(f"\nDropout = {dropout:.1f}:")
    print(f"  Variance entre passes: {variance:.6f}")

print(f"\n{'='*70}")
print("Observation: Plus de dropout = plus de variance!")
print(f"{'='*70}")

### Exercice 3: Visualiser l'Effet des Residual Connections

Comparez un bloc avec et sans residual connections.

In [ ]:
# TODO: Créez une version sans residual connections et comparez
class TransformerBlockNoResidual(nn.Module):
    """TransformerBlock SANS residual connections (pour comparaison)."""
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.attention = MultiHeadAttention(d_model, num_heads, dropout=dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.feed_forward = FeedForward(d_model, d_ff, dropout=dropout)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, mask=None):
        # SANS residual connections
        x = self.norm1(self.dropout(self.attention(x, mask)))
        x = self.norm2(self.dropout(self.feed_forward(x)))
        return x

# Comparer
d_model = 128
x = torch.randn(1, 10, d_model)

block_with_residual = TransformerBlock(d_model, 4, 512, dropout=0.0)
block_no_residual = TransformerBlockNoResidual(d_model, 4, 512, dropout=0.0)

block_with_residual.eval()
block_no_residual.eval()

with torch.no_grad():
    out_with = block_with_residual(x, mask=None)
    out_without = block_no_residual(x, mask=None)

print("=" * 70)
print("EXERCICE 3: Impact des Residual Connections")
print("=" * 70)
print(f"\nAvec residual connections:")
print(f"  Output mean: {out_with.mean().item():.6f}")
print(f"  Output std: {out_with.std().item():.6f}")

print(f"\nSans residual connections:")
print(f"  Output mean: {out_without.mean().item():.6f}")
print(f"  Output std: {out_without.std().item():.6f}")

print(f"\nObservation:")
print(f"  - Les residual connections stabilisent les activations")
print(f"  - Sans elles, les valeurs peuvent exploser ou disparaître")
print(f"  - Essentielles pour entraîner des réseaux profonds!")

## 8. Résumé et Points Clés

### Ce que nous avons appris

1. **Architecture du TransformerBlock:**
   - Deux sous-couches: Multi-Head Attention + Feed-Forward
   - Residual connections après chaque sous-couche
   - LayerNorm après chaque residual connection
   - Dropout pour la régularisation

2. **Connexions Résiduelles:**
   - Permettent au gradient de circuler directement
   - Essentielles pour entraîner des réseaux profonds
   - Formule: output = x + F(x)

3. **Layer Normalization:**
   - Normalise sur la dimension des features
   - Stabilise l'entraînement
   - Différent de BatchNorm

4. **BERT vs GPT:**
   - BERT: Pas de masque (bidirectionnel)
   - GPT: Masque causal (autorégressif)
   - Même architecture, usage différent

5. **Empiler des Blocs:**
   - Un transformer = Stack de N TransformerBlocks
   - Dimension préservée à chaque étape
   - Plus de blocs = Plus de capacité

### Rôle dans le Transformer Complet

```
Input Embeddings
      ↓
TransformerBlock 1
      ↓
TransformerBlock 2
      ↓
      ...
      ↓
TransformerBlock N
      ↓
Output Projection
```

### Prochaines Étapes

Dans le prochain notebook, nous allons:
1. Assembler le modèle Mini-GPT complet
2. Ajouter les embeddings d'entrée
3. Ajouter la projection de sortie
4. Créer un modèle de génération de texte fonctionnel

**Continuez vers:** `03_complete_mini_gpt.ipynb`

## 9. Références

### Articles Scientifiques

1. **Attention Is All You Need** (Vaswani et al., 2017)
   - Paper original des Transformers
   - Décrit l'architecture TransformerBlock

2. **Deep Residual Learning for Image Recognition** (He et al., 2015)
   - Introduction des residual connections
   - Inspiration pour les transformers

3. **Layer Normalization** (Ba et al., 2016)
   - Introduction de LayerNorm
   - Analyse théorique et empirique

4. **BERT: Pre-training of Deep Bidirectional Transformers** (Devlin et al., 2018)
   - Utilisation bidirectionnelle des transformers

5. **Language Models are Unsupervised Multitask Learners** (Radford et al., 2019)
   - GPT-2: Utilisation autoregressive des transformers

### Ressources Additionnelles

- PyTorch Documentation: https://pytorch.org/docs/stable/nn.html
- The Illustrated Transformer: http://jalammar.github.io/illustrated-transformer/
- Annotated Transformer: http://nlp.seas.harvard.edu/annotated-transformer/
- Understanding LSTM Networks: http://colah.github.io/posts/2015-08-Understanding-LSTMs/